In [35]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import norm # CDF norm dist
from datetime import datetime
import abc
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
sns.set_style('darkgrid')

In [36]:
TICKER = 'TSLA' # Tesla

# Options Pricing

This notebook explores key models and methodologies used in options pricing. Understanding and accurately estimating the fair value of options provides valuable insights for both risk management and strategy development in financial markets. Here, we dive into two foundational pricing models—Black-Scholes and the Binomial model, both of which have unique strengths and applications within the realm of derivatives pricing.

## 1. Overview

Options are financial derivatives that provide the right, but not the obligation, to buy or sell an underlying asset at a predetermined price, known as the strike price, on or before a specified expiration date. Options come in two main types: **calls**, which give the holder the right to buy, and **puts**, which give the holder the right to sell. The value of an option is composed of **intrinsic value** (the difference between the current asset price and the strike price) and **extrinsic value** (additional value attributed to time and volatility).

Options pricing is influenced by several key factors: **time to expiration** (the longer the time, the greater the potential for value changes), **volatility** of the underlying asset (higher volatility generally increases option prices), **interest rates** (affecting the cost of carrying options), and the **price of the underlying asset**. These factors, combined, impact an option's fair value and its behavior under changing market conditions, making options valuable tools for hedging and speculative strategies in financial markets.

In [37]:
def get_stock_vol(prices: pd.Series, trading_days:int=252):
    returns = prices / prices.shift(1)
    log_returns = np.log(returns)
    return log_returns.std() * (trading_days**0.5)

def get_option(symbol:str) -> tuple[pd.DataFrame, pd.DataFrame, datetime]:
    
    ticker = yf.Ticker(symbol)
    expiration_dates = ticker.options

    if len(expiration_dates) == 0:
        print(f'There are no listed options for {symbol}')
        return [], [], None

    options = ticker.option_chain(expiration_dates[0])
    return options.calls, options.puts, expiration_dates[0]

def get_period(date):
    date = datetime.strptime(date, '%Y-%m-%d')
    current_date = datetime.now()
    years_difference = (date - current_date).days / 365
    return years_difference

def get_option_chain(symbol):

    ticker = yf.Ticker(symbol)
    expiration_dates = ticker.options

    options_df = pd.DataFrame()

    for exp_dt in expiration_dates:
        option = ticker.option_chain(exp_dt)
        option = pd.DataFrame([*option.calls.values, *option.puts.values], columns=option.calls.columns)
        option['expiration_date'] = pd.to_datetime(exp_dt).date()
        option['days_to_expire'] = get_period(exp_dt)
        option['type'] = option['contractSymbol'].str[10].apply(lambda e: 'Call' if e == 'C' else 'Put')
        options_df = pd.concat([options_df, option])

    return options_df

def get_treasury_10():
    today = datetime.now()
    start_date = today.replace(year=today.year-10)
    data = yf.download('^TNX', start=start_date)
    return data['Close'].iloc[-1] / 100


In [38]:
closes = yf.download(TICKER)['Close']
options = get_option_chain(TICKER)
options.head(3)

[*********************100%%**********************]  1 of 1 completed


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,expiration_date,days_to_expire,type
0,TSLA241122C00075000,2024-11-18 14:30:02+00:00,75.0,264.45,262.25,265.35,-2.849976,-1.066209,3.0,5.0,5.109379,True,REGULAR,USD,2024-11-22,0.008219,Call
1,TSLA241122C00080000,2024-11-14 18:48:33+00:00,80.0,234.18,257.15,260.35,0.000000,0.000000,1.0,3.0,4.187505,True,REGULAR,USD,2024-11-22,0.008219,Call
2,TSLA241122C00085000,2024-11-11 20:35:40+00:00,85.0,263.50,252.15,255.35,0.000000,0.000000,2.0,2.0,4.062505,True,REGULAR,USD,2024-11-22,0.008219,Call


## 1.1 Options Analysis

### 1.1.1 Implied Volatility

In [50]:
call_options = options[options['type'] == 'Call']
fig = go.Figure()

for expiration, group in call_options.groupby('expiration_date'):
    fig.add_trace(go.Scatter(
        x=group['strike'],
        y=group['impliedVolatility'],
        mode='markers+lines',
        name=str(expiration)
    ))

# Add the current stock price
fig.add_vline(closes.iloc[-1], 
              line_dash='dash',
              annotation_text=f"At The Money Price", 
              annotation_position="top right",
              )

fig.update_layout(
    title="Implied Volatility by Strike Price for Each Expiration Date",
    xaxis_title="Strike Price",
    yaxis_title="Implied Volatility",
    legend_title="Expiration Date",
    template="plotly_white"
)


## 2. The Greeks

The Greeks are key metrics that measure different risks in options trading. They can be used to manage exposure to various factors, ensuring that risks stay within acceptable levels. 

### 2.1. Delta (Δ)

Measures the rate of change of an option's price with respect to changes in the price of the underlying asset. In other words, it represents how much the price of the option is expected to change for a small change in the underlying asset's price.

In [40]:
def delta(d1:float, is_call:bool):
    signal = 1 if is_call else -1
    return signal*norm.cdf(signal*d1, 0.0, 1.0)

### 2.2 Gamma (Γ)

Measures the rate of change of Delta with respect to changes in the price of the underlying asset. It indicates how much Delta will change when the price of the underlying asset changes. Gamma is important for understanding the curvature of the option's price relative to changes in the underlying asset's price.

In [41]:
def gamma(d1, S, sigma, T):
        return norm.pdf(d1, 0.0, 1.0) / (S * sigma * np.sqrt(T))

### 2.3 Theta (Θ) or Time Decay

Measures the rate of change of the option's price with respect to time. Theta is negative for **call** and **put** options, meaning that as time passes, the option's value decreases.

In [42]:
def theta(d1, d2, S, sigma, T, r, K, is_call):
    signal = 1 if is_call else -1
    return (-S * norm.pdf(d1, 0.0, 1.0) * sigma / (2 * np.sqrt(T)) - signal * r * K * np.exp(-r * T) * norm.cdf(signal*d2, 0.0, 1.0))

### 2.4 Vega (V)

Measures the sensitivity of the option's price to changes in the volatility of the underlying asset. Vega is always positive for both **call** and **put** options, indicating that an increase in volatility increases the price of the option.

In [43]:
def vega(d1, S, T):
        return S * norm.pdf(d1, 0.0, 1.0) * np.sqrt(T)

### 2.5 Rho (ρ)

Measures the sensitivity of the option's price to changes in the risk-free interest rate. Rho is positive for **call** options and negative for **put** options. This means that for call options, as interest rates rise, the price of the option increases, and for put options, as interest rates rise, the price decreases.

In [44]:
def rho(d2, K, T, r, is_call):
        signal = 1 if is_call else -1
        return signal * K * T * np.exp(-r * T) * norm.cdf(signal*d2, 0.0, 1.0)

## Pricinng Models

In [45]:
class OptionPricingBaseModel:

    """
    Abstract base class for option pricing models, providing a structure for 
    calculating call and put option prices and their associated Greeks.

    Attributes:
    - S (float): Underlying asset price.
    - K (float): Strike price.
    - r (float): Risk-free interest rate (annualized).
    - sigma (float): Volatility of the underlying asset.
    - T (float): Time to expiration in years.
    - expiration_date (datetime, optional): The expiration date of the option (used if T is not provided).
    
    Methods:
    - get_call_price(): Abstract method to calculate call option price.
    - get_put_price(): Abstract method to calculate put option price.
    - get_call_greeks(): Calculates and returns the Greeks for a call option.
    - get_put_greeks(): Calculates and returns the Greeks for a put option.
    - get_call(call): Calculates the price and Greeks for a call option.
    - get_put(put): Calculates the price and Greeks for a put option.
    """

    def __init__(self, S:float, K:float, r:float, sigma:float, T=None, expiration_date=None):
        assert T is not None or expiration_date is not None, \
        'You must specify either the option expiration date or the time to expire (in years)'

        self.S = S # Underlying price
        self.K = K # Strike
        self.r = r # Risk-free rate
        self.sigma = sigma  # Volatility of the underlying asset
        self.T = T or get_period(expiration_date)
    
    def get_call(self):
        price = self.get_call_price()
        greeks = self.get_call_greeks()
        return pd.Series({f'price': price, **greeks}).add_prefix(self._model_prefix)
    
    def get_put(self):
        price = self.get_put_price()
        greeks = self.get_put_greeks()
        return pd.Series({f'price': price, **greeks}).add_prefix(self._model_prefix)

    @property 
    @abc.abstractmethod
    def _model_prefix(self) -> str:
        pass

    @abc.abstractmethod
    def get_call_price(self):
        pass

    @abc.abstractmethod
    def get_put_price(self):
        pass

    @abc.abstractmethod
    def _get_greeks(self, is_call) -> dict:        
        pass

    def get_call_greeks(self) -> dict:
        return self._get_greeks(is_call=True)

    def get_put_greeks(self) -> dict:
        return self._get_greeks(is_call=False)


# Black-Scholes

The Black-Scholes Model is a solution for pricing European-style options. This model assumes a constant volatility and a frictionless market, allowing us to calculate the theoretical price of options based on the underlying asset's current price, strike price, time to expiration, risk-free rate, and volatility. While the Black-Scholes Model is highly efficient and widely used in the industry, it does have some limitations, particularly in accounting for changes in volatility over time or the early exercise feature of American options.

**Assumptions**

- Constant Volatility
- No Dividends
- Log-Normal Distribution of Asset Prices
- European Option
- No Transaction Costs
- Risk-Free Rate is Constant

In [46]:
class BlackScholes(OptionPricingBaseModel):

    def __init__(self, S:float, K:float, r:float, sigma:float, T=None, expiration_date=None):
        assert T is not None or expiration_date is not None, \
        'You must specify either the option expiration date or the time to expire (in years)'
        super().__init__(S, K, r,sigma, T, expiration_date)

        self.S = S # Underlying price
        self.K = K # Strike
        self.r = r # Risk-free rate
        self.sigma = sigma  # Volatility of the underlying asset
        self.T = T or get_period(expiration_date)
    
    @property
    def d1(self):
        return (np.log(self.S / self.K) + (self.r + 0.5 * self.sigma ** 2) * self.T) / (self.sigma * np.sqrt(self.T))

    @property
    def d2(self):
        return self.d1 - self.sigma * np.sqrt(self.T)

    def get_call_price(self):
        return (self.S * norm.cdf(self.d1, 0.0, 1.0) - self.K * np.exp(-self.r * self.T) * norm.cdf(self.d2, 0.0, 1.0))
    
    def get_put_price(self):
        return (self.K * np.exp(-self.r * self.T) * norm.cdf(-self.d2, 0.0, 1.0) - self.S * norm.cdf(-self.d1, 0.0, 1.0))

    def _get_greeks(self, is_call):
        return {
            'delta': delta(self.d1, is_call),
            'gamma': gamma(self.d1, self.S, self.sigma, self.T),
            'theta': theta(self.d1, self.d2, self.S, self.sigma, self.T, self.r, self.K, is_call),
            'vega': vega(self.d1, self.S, self.T),
            'rho':  rho(self.d1, self.K, self.T, self.r, is_call)
        }
    
    @property
    def _model_prefix(self): return 'bsm_'


In [47]:
calls, puts, date = get_option(TICKER)
closes = yf.download(TICKER)['Close']

r = get_treasury_10()
S = closes.iloc[-1]
sigma = get_stock_vol(closes)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [48]:
bsm_calls_df = calls.apply(lambda x: BlackScholes(S=S, K=x['strike'], r=r, sigma=sigma, expiration_date=date).get_call(), axis=1)
calls = pd.concat([calls, bsm_calls_df], axis=1)
calls.head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,bsm_price,bsm_delta,bsm_gamma,bsm_theta,bsm_vega,bsm_rho
0,TSLA241122C00075000,2024-11-18 14:30:02+00:00,75.0,264.45,262.25,265.35,-2.849976,-1.066209,3.0,5,5.109379,True,REGULAR,USD,263.767195,1.0,5.915805e-187,-3.309299,3.186091e-184,0.616215
1,TSLA241122C00080000,2024-11-14 18:48:33+00:00,80.0,234.18,257.15,260.35,0.000000,0.000000,1.0,3,4.187505,True,REGULAR,USD,258.769009,1.0,1.655542e-171,-3.529919,8.916296e-169,0.657296
2,TSLA241122C00085000,2024-11-11 20:35:40+00:00,85.0,263.50,252.15,255.35,0.000000,0.000000,2.0,2,4.062505,True,REGULAR,USD,253.770822,1.0,1.301264e-157,-3.750539,7.008252e-155,0.698377
3,TSLA241122C00090000,2024-10-07 18:52:43+00:00,90.0,152.80,205.30,207.55,0.000000,0.000000,NaN,4,0.000010,True,REGULAR,USD,248.772636,1.0,4.676392e-145,-3.971159,2.518577e-142,0.739458
4,TSLA241122C00095000,2024-11-14 19:39:03+00:00,95.0,218.61,242.25,245.40,0.000000,0.000000,1.0,3,4.484379,True,REGULAR,USD,243.774449,1.0,1.146165e-133,-4.191779,6.172933e-131,0.780539


In [49]:
bsm_puts_df = calls.apply(lambda x: BlackScholes(S=S, K=x['strike'], r=r, sigma=sigma, expiration_date=date).get_put(), axis=1)
puts = pd.concat([puts, bsm_puts_df], axis=1)
puts.head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,bsm_price,bsm_delta,bsm_gamma,bsm_theta,bsm_vega,bsm_rho
0,TSLA241122P00075000,2024-11-18 17:59:25+00:00,75.0,0.01,0.0,0.01,0.0,0.0,19.0,549.0,4.125005,False,REGULAR,USD,2.136691e-187,-3.554243e-187,5.915805e-187,-1.106302e-182,3.186091e-184,-2.190177e-187
1,TSLA241122P00080000,2024-11-14 14:52:06+00:00,80.0,0.01,0.0,0.01,0.0,0.0,3.0,586.0,4.000005,False,REGULAR,USD,6.524069e-172,-1.038974e-171,1.655542e-171,-3.095927e-167,8.916296e-169,-6.829130e-172
2,TSLA241122P00085000,2024-11-15 19:34:43+00:00,85.0,0.01,0.0,0.01,0.0,0.0,90.0,771.0,3.875000,False,REGULAR,USD,5.585483e-158,-8.523057e-158,1.301264e-157,-2.433360e-153,7.008252e-155,-5.952305e-158
3,TSLA241122P00090000,2024-11-18 18:04:04+00:00,90.0,0.01,0.0,0.01,0.0,0.0,1.0,9979.0,3.687501,False,REGULAR,USD,2.183281e-145,-3.194486e-145,4.676392e-145,-8.744642e-141,2.518577e-142,-2.362187e-145
4,TSLA241122P00095000,2024-11-18 18:03:39+00:00,95.0,0.01,0.0,0.01,0.0,0.0,1.0,5771.0,3.500001,False,REGULAR,USD,5.813532e-134,-8.161015e-134,1.146165e-133,-2.143228e-129,6.172933e-131,-6.369988e-134
